# Signature Optimal Stopping (Rust-backed)\n

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
np.random.seed(0)
t = np.linspace(0,1,200)
x = np.cumsum(np.random.normal(0,0.02,200)) + 1.0
path = list(zip(t.tolist(), x.tolist()))
try:
    import hft_py
    stop = hft_py.signature_opt_stop_py(path, 0.9)
    print('stop:', stop)
except Exception as e:
    stop = None
    print('Rust binding not available:', e)
df = pd.DataFrame({'t':t,'x':x}); sns.lineplot(data=df, x='t', y='x')
if stop is not None:
    st, sx = stop; plt.scatter([st],[sx], color='red', s=60, label='stop')
plt.legend(); plt.show()
